# 이것은 데이터 제출을 위해서 만들어진 코드입니다

## 대학 컬럼을 넣는 과정은 생략되었습니다. 이미 제가 만든 대학 컬럼까지 만든 것을 읽어왔습니다.

## 이것이 최선은 아닙니다. voting regressor에서 n_iter였나(?)라는 parameter를 늘리면 더욱 많은 sub model들을 탐색할 수 있을 거에요. 그리고 모델 자체를 더 많이 고려할 수도 있죠.

## 그러나 오늘 일단 제출을 목표로 2개 모델만으로 voting regressor가 되었습니다. 이것을 기준으로 모델을 더욱 많이 고려할 수도 있겠습니다.

## 모델들이 돌아갈 조건이 다 다릅니다. 웬만하면 돌아가는 모델, nan값이 없어야하는모델, standardization이 필요한 모델, one hot encoding으로 되어 모든 값들이 정수여야 하는 모델들까지. 그래서 오늘은 가장 조건이 어렵지 않은모델  2개를 voting에 부쳤습니다.

## In[ ] 번호가 1번부터 정렬 안된 까닭은 이거 한번 돌리는데 엄청 오래 걸리기 때문입니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_num = pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/modeling_data_withuniversity.csv',encoding = 'EUC-KR',index_col=0)

C:\Users\drlim\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df_num.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,total_tourist_groupby,university_count
0,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,8.806154,False,1.562276e+13,42767.077,1.980289e+06,18
1,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,8,11.461487,False,1.562276e+13,42767.077,1.980289e+06,18
2,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,8,11.461487,False,1.562276e+13,42767.077,1.980289e+06,18
3,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,11,11.461487,False,1.562276e+13,42767.077,1.980289e+06,18
4,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,0,7.066787,True,1.562276e+13,42767.077,NaN,18


### 그나마 돌릴 수 있는 조건이 덜 까다로운 LGboost, Adaboost를 하려 하는데 얘네들은 NAN을 허락하지 않는다. (왜?)

### 그래서 NA가 있는 컬럼이 두개인데 이 중에서 NA가 너무 많아 처치 불가인 colum인 total_tourist_grpby는 drop, NAN값이 많지 않은 total_sale_by_indst 는 column mean으로 채운다!

In [4]:
df_num.drop(columns= ['total_tourist_groupby'],inplace=True)

## 맨 먼저 카테고리변수들 labelencoding하면 편함! 말 그대로 글자로 되어 있는것들

In [5]:
from sklearn.preprocessing import LabelEncoder
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid'] # 말 그대로 
# here, don't apply FLC!
df_num[cat_features]= df_num[cat_features].apply(LabelEncoder().fit_transform)

In [6]:
df_num.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,university_count
0,0,0,0,1,1,1,843100.0,11,2019,0,8.806154,0,1.562276e+13,42767.077,18
1,0,0,0,1,1,1,139000.0,6,2019,8,11.461487,0,1.562276e+13,42767.077,18
2,0,0,0,1,1,1,27500.0,8,2019,8,11.461487,0,1.562276e+13,42767.077,18
3,0,0,0,1,1,1,395500.0,9,2019,11,11.461487,0,1.562276e+13,42767.077,18
4,0,0,0,1,1,1,427510.0,3,2020,0,7.066787,1,1.562276e+13,42767.077,18


In [7]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    df_num[ca] = df_num[ca].astype('category')

In [8]:
df_num.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                    category
FLC                            category
AMT                             float64
month                             int64
year                              int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
university_count                  int64
dtype: object

In [9]:
train_num = df_num.sample(frac=1, random_state=2020)
train_features = train_num.drop(['AMT'], axis=1) #1057394 14
train_target = np.log1p(train_num['AMT']) #1057394,

In [10]:
train_features.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                    category
FLC                            category
month                             int64
year                              int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
university_count                  int64
dtype: object

In [11]:
train_target.dtypes

dtype('float64')

In [12]:
x = train_features
y = train_target

In [13]:
k = int(len(x)*0.7)

In [14]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [15]:
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

In [16]:
x_train.columns == x_val.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [17]:
x_train.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                    category
FLC                            category
month                             int64
year                              int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
university_count                  int64
dtype: object

# Voting Regressor부분만 적습니다

모델은 4개 돌릴수 있겠지만 각 모델 다 조건이 다릅니다

(standardization, 모두 다 one hot encoding, na만없으면돼 등등)

lgbm, adaboost는 nan만 없으면 되어서 수행!

In [18]:
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, VotingRegressor, GradientBoostingRegressor

In [19]:
r1 = GradientBoostingRegressor(random_state=2020,n_estimators=400, learning_rate = 0.01, max_depth = 13, max_features = 'sqrt', loss='huber', subsample=0.7)
r2 = LGBMRegressor(learning_rate=0.1,boosting_type='gbdt',random_state=2020,n_estimators=400, objective = 'regression', importance_type = 'split', num_leaves = 130)
r3 = xgb.XGBRegressor(learning_rate=0.01,objective='reg:squarederror',random_state = 2020, n_estimators = 400,subsample = 0.7, booster='gbtree')
r4 = AdaBoostRegressor(random_state = 2020, n_estimators = 400, learning_rate = .01, loss='exponential')
r5 = RandomForestRegressor(n_estimators = 400, max_depth =15, min_samples_split = 5, min_samples_leaf =5, oob_score = True, random_state= 2020)

In [20]:
vreg= VotingRegressor([('lgb', r2),('adb',r4),('RF',r5)])

In [21]:
print(x_val.shape)
print(x_train.shape)
print(y_val.shape)
print(y_train.shape)

(317219, 14)
(740175, 14)
(317219,)
(740175,)


In [22]:
x_train.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,university_count
816916,13,17,16,4,2,4,7,2019,1,11.461490,0,6.011270e+13,10972.310,19
698999,11,16,5,5,1,5,10,2019,0,7.066787,0,4.656089e+13,251207.500,7
941132,15,2,13,6,2,5,1,2020,1,7.066787,0,1.301312e+12,155366.000,21
100207,1,21,8,2,1,1,3,2020,0,5.297286,1,1.076278e+15,42767.077,61
603857,8,38,11,1,1,1,5,2019,7,7.066787,0,5.395069e+14,6057877.000,48


In [23]:
vreg = vreg.fit(x_train, y_train)

ValueError: Input contains NaN

## 예전에 수치형변수들도 다 integer coding했을 때는 이런 에러가 뜨지 않았는데 이번에 수치형변수 카테고리변수 잘 구별했을 때는 에러가 뜬다!

##  NaN 값 imputing 필요!


In [24]:
x.isna().sum()

CARD_SIDO_NM                       0
STD_CLSS_NM                        0
HOM_SIDO_NM                        0
AGE                                0
SEX_CTGO_CD                        0
FLC                                0
month                              0
year                               0
festival_count                     0
rain_day                           0
covid                              0
age_population                     0
total_sale_by_indst_groupby    17378
university_count                   0
dtype: int64

### Total Sale by industry groupby 열 : '지역', '산업'으로 묶여있다!

### 지역보다 산업이 더욱 variation이 크므로 

### missing 데이터는 해당 산업의 median으로 impute!

In [25]:
med = x.total_sale_by_indst_groupby.median()
x['total_sale_by_indst_groupby'] = x.total_sale_by_indst_groupby.fillna(med)

In [26]:
x['total_sale_by_indst_groupby'].value_counts()

10972.310      98606
251207.500     95839
1253926.000    94321
42767.080      93628
98196.390      92431
442858.800     92410
155366.000     91423
6057877.000    91164
1976481.000    88523
721586.000     82682
251207.511     28506
721585.993     22878
1976481.108    18169
98196.393      13047
155366.044     10323
42767.077      10221
442858.791      9966
6057876.648     9050
1253926.239     7398
10972.309       6809
Name: total_sale_by_indst_groupby, dtype: int64

In [27]:
x.isna().sum()

CARD_SIDO_NM                   0
STD_CLSS_NM                    0
HOM_SIDO_NM                    0
AGE                            0
SEX_CTGO_CD                    0
FLC                            0
month                          0
year                           0
festival_count                 0
rain_day                       0
covid                          0
age_population                 0
total_sale_by_indst_groupby    0
university_count               0
dtype: int64

In [28]:
k = int(len(x)*0.7)
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [29]:
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

바로 아래 코드가 저한테는 1시간 15분 걸렸습니다! 

In [30]:
vreg = vreg.fit(x_train, y_train)

In [31]:
y_train_predict = vreg.predict(x_train) #prediction inside predicted data. can compute 'TRAINING RMSLE'

In [32]:
y_val_pred = vreg.predict(x_val) # prediction for validation data. can compute 'validation RMSLE'

여기서 sklearn에서 import하는 mean_squared_log_error랑 데이터콘에서 쓰는 rmsle랑 똑같은지는 잘 모르겠기는 합니다만 중요성은 없습니다

In [49]:
from sklearn.metrics import mean_squared_log_error
RMSLE_train = mean_squared_log_error(y_train,y_train_predict)
RMSLE_val =  mean_squared_log_error(y_val,y_val_pred)

In [50]:
RMSLE_train

0.010543181240651933

In [51]:
RMSLE_val

0.01059157160390221

# 이제 진짜 예측!

In [33]:
temp_withuniversity =  pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/template_withuniversity.csv',encoding = 'EUC-KR',index_col=0)

C:\Users\drlim\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [34]:
temp_withuniversity

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,rain_day,age_population,total_sale_by_indst_groupby,total_tourist_groupby,home_ratio,covid,festival_count,university_count
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0,18
1,강원,건강보조식품 소매업,강원,20s,1,1,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0,18
2,강원,건강보조식품 소매업,강원,20s,1,2,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0,18
3,강원,건강보조식품 소매업,강원,20s,1,2,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0,18
4,강원,건강보조식품 소매업,강원,20s,1,3,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0,18
5,강원,건강보조식품 소매업,강원,20s,1,3,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0,18
6,강원,건강보조식품 소매업,강원,20s,1,4,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0,18
7,강원,건강보조식품 소매업,강원,20s,1,4,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0,18
8,강원,건강보조식품 소매업,강원,20s,1,5,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0,18
9,강원,건강보조식품 소매업,강원,20s,1,5,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0,18


# 역시 Test data (template)는 정리가 안되었으니 feature를 train data에 맞게 변경!

In [35]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid']
# here, don't apply FLC!
temp_withuniversity[cat_features]= temp_withuniversity[cat_features].apply(LabelEncoder().fit_transform)

In [36]:
temp_withuniversity

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,rain_day,age_population,total_sale_by_indst_groupby,total_tourist_groupby,home_ratio,covid,festival_count,university_count
0,0,0,0,1,1,1,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,0,0,18
1,0,0,0,1,1,1,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,0,0,18
2,0,0,0,1,1,2,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,0,0,18
3,0,0,0,1,1,2,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,0,0,18
4,0,0,0,1,1,3,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,0,0,18
5,0,0,0,1,1,3,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,0,0,18
6,0,0,0,1,1,4,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,0,0,18
7,0,0,0,1,1,4,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,0,0,18
8,0,0,0,1,1,5,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,0,0,18
9,0,0,0,1,1,5,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,0,0,18


In [37]:
temp_withuniversity.drop(columns= ['total_tourist_groupby'],inplace=True)

In [38]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    temp_withuniversity[ca] = temp_withuniversity[ca].astype('category')

In [39]:
temp_withuniversity.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                    category
FLC                            category
month                             int64
year                              int64
rain_day                        float64
age_population                    int64
total_sale_by_indst_groupby     float64
home_ratio                      float64
covid                          category
festival_count                    int64
university_count                  int64
dtype: object

In [40]:
med2 = temp_withuniversity.total_sale_by_indst_groupby.median()
temp_withuniversity['total_sale_by_indst_groupby'] = temp_withuniversity.total_sale_by_indst_groupby.fillna(med)

In [41]:
temp_withuniversity.isna().sum() #OKAY

CARD_SIDO_NM                   0
STD_CLSS_NM                    0
HOM_SIDO_NM                    0
AGE                            0
SEX_CTGO_CD                    0
FLC                            0
month                          0
year                           0
rain_day                       0
age_population                 0
total_sale_by_indst_groupby    0
home_ratio                     0
covid                          0
festival_count                 0
university_count               0
dtype: int64

In [42]:
temp_withuniversity.drop(columns='home_ratio',inplace=True)

예상 못했던  home ratio 컬럼도 삭제하니
이제 PREDICTED 값 넣을 준비 모두 완료

아래 명령어도 수행되는데 25분 넘게 걸림

In [43]:
pred = vreg.predict(temp_withuniversity)

In [44]:
pred = np.exp(pred)

In [45]:
temp_withuniversity['AMT'] = np.round(pred,0)

In [46]:
temp_withuniversity['REG_YYMM'] = temp_withuniversity['year']*100 + temp_withuniversity['month']

In [47]:
temp_withuniversity.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,rain_day,age_population,total_sale_by_indst_groupby,covid,festival_count,university_count,AMT,REG_YYMM
0,0,0,0,1,1,1,4,2020,8.806154,15622761472137,42767.077,0,0,18,277869.0,202004
1,0,0,0,1,1,1,7,2020,11.461487,15622761472137,42767.077,0,0,18,288869.0,202007
2,0,0,0,1,1,2,4,2020,8.806154,15622761472137,42767.077,0,0,18,166194.0,202004
3,0,0,0,1,1,2,7,2020,11.461487,15622761472137,42767.077,0,0,18,173886.0,202007
4,0,0,0,1,1,3,4,2020,8.806154,15622761472137,42767.077,0,0,18,181529.0,202004


In [48]:
temp_withuniversity = temp_withuniversity[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

In [49]:
temp_withuniversity.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,277869.0
1,202007,0,0,288869.0
2,202004,0,0,166194.0
3,202007,0,0,173886.0
4,202004,0,0,181529.0


In [50]:
temp_withuniversity = temp_withuniversity.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [51]:
temp_withuniversity

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,262950918.0
1,202004,0,1,407116014.0
2,202004,0,2,378889850.0
3,202004,0,3,150192875.0
4,202004,0,4,110391182.0
5,202004,0,5,102113860.0
6,202004,0,6,238289712.0
7,202004,0,7,336903340.0
8,202004,0,8,135478137.0
9,202004,0,9,352678018.0


# 이제 마지막 남은 것은 CARD_SIDO_NM, STD_CLSS_NM을 원래 클래스로 복구시키는것이다!

어떻게 encoding된건가?

먼저 정수인코딩된 거

In [52]:
df_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,university_count
0,0,0,0,1,1,1,843100.0,11,2019,0,8.806154,0,1.562276e+13,42767.077,18
1,0,0,0,1,1,1,139000.0,6,2019,8,11.461487,0,1.562276e+13,42767.077,18
2,0,0,0,1,1,1,27500.0,8,2019,8,11.461487,0,1.562276e+13,42767.077,18
3,0,0,0,1,1,1,395500.0,9,2019,11,11.461487,0,1.562276e+13,42767.077,18
4,0,0,0,1,1,1,427510.0,3,2020,0,7.066787,1,1.562276e+13,42767.077,18
5,0,0,0,1,1,1,605000.0,2,2019,4,5.297286,0,1.562276e+13,42767.077,18
6,0,0,0,1,1,1,311200.0,1,2019,4,5.297286,0,1.562276e+13,42767.077,18
7,0,0,0,1,1,1,168500.0,1,2020,0,5.297286,0,1.562276e+13,42767.077,18
8,0,0,0,1,2,1,752646.0,1,2020,0,5.297286,0,1.562276e+13,42767.077,18
9,0,0,0,1,2,1,893400.0,6,2019,8,11.461487,0,1.562276e+13,42767.077,18


위에것과 아래거(한글 쓰인거) 비교

In [53]:
df_num2 = pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/modeling_data.csv',encoding = 'EUC-KR',index_col=0)

C:\Users\drlim\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [54]:
df_num2

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
1,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,0,11,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
5,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,1,0,7.066787,True,1.562276e+13,42767.077,10.591,NaN
6,강원,건강보조식품 소매업,강원,20s,1,1,605000.0,2,2019,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
7,강원,건강보조식품 소매업,강원,20s,1,1,311200.0,1,2019,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
8,강원,건강보조식품 소매업,강원,20s,1,1,168500.0,1,2020,0,0,5.297286,False,1.562276e+13,42767.077,10.591,NaN
9,강원,건강보조식품 소매업,강원,20s,2,1,752646.0,1,2020,0,0,5.297286,False,1.562276e+13,42767.077,10.591,NaN
10,강원,건강보조식품 소매업,강원,20s,2,1,893400.0,6,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06


In [55]:
temp_withuniversity['CARD_SIDO_NM'] = temp_withuniversity.CARD_SIDO_NM.replace(np.arange(17) , ['강원','경기','경남','경북','광주','대구','대전','부산','서울','세종','울산','인천','전남','전북','제주','충남','충북'])

In [56]:
temp_withuniversity

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,0,262950918.0
1,202004,강원,1,407116014.0
2,202004,강원,2,378889850.0
3,202004,강원,3,150192875.0
4,202004,강원,4,110391182.0
5,202004,강원,5,102113860.0
6,202004,강원,6,238289712.0
7,202004,강원,7,336903340.0
8,202004,강원,8,135478137.0
9,202004,강원,9,352678018.0


위처럼 노가다 할 거 없이 아래처럼 하면 순서 잘 정렬되어있음.

In [57]:
df_num2['STD_CLSS_NM'].unique()

array(['건강보조식품 소매업', '골프장 운영업', '과실 및 채소 소매업', '관광 민예품 및 선물용품 소매업',
       '그외 기타 분류안된 오락관련 서비스업', '그외 기타 스포츠시설 운영업', '그외 기타 종합 소매업',
       '기타 대형 종합 소매업', '기타 수상오락 서비스업', '기타 외국식 음식점업', '기타 주점업',
       '기타음식료품위주종합소매업', '내항 여객 운송업', '마사지업', '면세점', '버스 운송업', '비알콜 음료점업',
       '빵 및 과자류 소매업', '서양식 음식점업', '수산물 소매업', '슈퍼마켓', '스포츠 및 레크레이션 용품 임대업',
       '여관업', '여행사업', '욕탕업', '육류 소매업', '일반유흥 주점업', '일식 음식점업', '자동차 임대업',
       '전시 및 행사 대행업', '중식 음식점업', '차량용 가스 충전업', '차량용 주유소 운영업', '체인화 편의점',
       '택시 운송업', '피자 햄버거 샌드위치 및 유사 음식점업', '한식 음식점업', '호텔업',
       '화장품 및 방향제 소매업', '휴양콘도 운영업', '정기 항공 운송업'], dtype=object)

In [58]:
df_num2['STD_CLSS_NM'].nunique()

41

In [73]:
class_list= np.append(df_num2['STD_CLSS_NM'].unique()[:30], ['정기 항공 운송업','중식 음식점업','차량용 가스 충전업','차량용 주유소 운영업','체인화 편의점','택시 운송업','피자 햄버거 샌드위치 및 유사 음식점업','한식 음식점업','호텔업','화장품 및 방향제 소매업','휴양콘도 운영업'])

In [74]:
temp_withuniversity['STD_CLSS_NM'] = temp_withuniversity.STD_CLSS_NM.replace(np.arange(41) ,class_list)

In [75]:
temp_withuniversity

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,262950918.0
1,1,202004,강원,골프장 운영업,407116014.0
2,2,202004,강원,과실 및 채소 소매업,378889850.0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,150192875.0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,110391182.0
5,5,202004,강원,그외 기타 스포츠시설 운영업,102113860.0
6,6,202004,강원,그외 기타 종합 소매업,238289712.0
7,7,202004,강원,기타 대형 종합 소매업,336903340.0
8,8,202004,강원,기타 수상오락 서비스업,135478137.0
9,9,202004,강원,기타 외국식 음식점업,352678018.0


마지막 남은 것은 index column을 id라는 이름의 normal column으로 바꾸는것뿐

In [76]:
temp_withuniversity.reset_index(level=0, inplace=True)

In [77]:
temp_withuniversity

,index,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,0,202004,강원,건강보조식품 소매업,262950918.0
1,1,1,202004,강원,골프장 운영업,407116014.0
2,2,2,202004,강원,과실 및 채소 소매업,378889850.0
3,3,3,202004,강원,관광 민예품 및 선물용품 소매업,150192875.0
4,4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,110391182.0
5,5,5,202004,강원,그외 기타 스포츠시설 운영업,102113860.0
6,6,6,202004,강원,그외 기타 종합 소매업,238289712.0
7,7,7,202004,강원,기타 대형 종합 소매업,336903340.0
8,8,8,202004,강원,기타 수상오락 서비스업,135478137.0
9,9,9,202004,강원,기타 외국식 음식점업,352678018.0


In [78]:
temp_withuniversity.rename(columns = {'index' : 'id'}, inplace = True)


In [79]:
temp_withuniversity

,id,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,0,202004,강원,건강보조식품 소매업,262950918.0
1,1,1,202004,강원,골프장 운영업,407116014.0
2,2,2,202004,강원,과실 및 채소 소매업,378889850.0
3,3,3,202004,강원,관광 민예품 및 선물용품 소매업,150192875.0
4,4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,110391182.0
5,5,5,202004,강원,그외 기타 스포츠시설 운영업,102113860.0
6,6,6,202004,강원,그외 기타 종합 소매업,238289712.0
7,7,7,202004,강원,기타 대형 종합 소매업,336903340.0
8,8,8,202004,강원,기타 수상오락 서비스업,135478137.0
9,9,9,202004,강원,기타 외국식 음식점업,352678018.0


In [81]:
temp_withuniversity.to_csv('C:/workspace/jeju/2020_jeju_creditcard/data/submission3.csv',index=False,encoding = 'EUC-KR')